This notebook will be used primarly for the Coursera Capstone project to predict traffic accident severity in Seattle. 

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

@hidden cell
#Code removed since it included credentials. Loaded the data from IBM account.

df_data_2 = pd.read_csv(body)
df_data_2.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [82]:
df=df_data_2
df.shape

(194673, 38)

First is being sure there are no rows with blank target values (SEVERITYCODE).

In [83]:
df['SEVERITYCODE'].dropna(axis=0,inplace=True)
df.groupby(['SEVERITYCODE']).count()

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
SEVERITYCODE,,,,,,,,,,,,,,,,,,,,,
1,132221,132221,136485,136485,136485,136485,136485,134750,37251,134081,...,132533,132405,460,81429,5802,136480,132622,136485,136485,136485
2,57118,57118,58188,58188,58188,58188,58188,57997,27819,57915,...,57128,57098,4207,33507,3531,58175,57147,58188,58188,58188


Next is to reduce the dataset to the relevant columns, i.e. those that help to predict the severity of accidents related to time of day, weather, and distracted driving. Not all columns will be used in the final analysis but will help to refine the data further. 

In [84]:
dfr=df[['SEVERITYCODE','INCDTTM','UNDERINFL','WEATHER','ROADCOND','SPEEDING',
        'INATTENTIONIND','HITPARKEDCAR','COLLISIONTYPE']].copy()
dfr.head()

,SEVERITYCODE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,SPEEDING,INATTENTIONIND,HITPARKEDCAR,COLLISIONTYPE
0,2,3/27/2013 2:54:00 PM,N,Overcast,Wet,NaN,NaN,N,Angles
1,1,12/20/2006 6:55:00 PM,0,Raining,Wet,NaN,NaN,N,Sideswipe
2,1,11/18/2004 10:20:00 AM,0,Overcast,Dry,NaN,NaN,N,Parked Car
3,1,3/29/2013 9:26:00 AM,N,Clear,Dry,NaN,NaN,N,Other
4,2,1/28/2004 8:04:00 AM,0,Raining,Wet,NaN,NaN,N,Angles


Business X would like to focus their analysis on traffic accidents involving motor vehicles since the vast majority of their employees commute by car. 

In [85]:
dfr['COLLISIONTYPE'].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

Accidents involving pedestrians and bicycles will be removed.

In [86]:
dfr=dfr.loc[dfr['COLLISIONTYPE']!='Pedestrian']
dfr=dfr.loc[dfr['COLLISIONTYPE']!='Cycles']
dfr['COLLISIONTYPE'].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

Other "human factors" such as speeding and hitting parked cars will also be eliminated from the dataset to remove confounding variables on the severity of the accident. 

In [87]:
dfr['SPEEDING'].value_counts()

Y    9161
Name: SPEEDING, dtype: int64

In [88]:
dfr=dfr.loc[dfr['SPEEDING']!='Y']
dfr['SPEEDING'].value_counts()

Series([], Name: SPEEDING, dtype: int64)

In [89]:
dfr['HITPARKEDCAR'].value_counts()

N    166415
Y      7074
Name: HITPARKEDCAR, dtype: int64

In [90]:
dfr=dfr.loc[dfr['HITPARKEDCAR']!='Y']
dfr['HITPARKEDCAR'].value_counts()

N    166415
Name: HITPARKEDCAR, dtype: int64

In [91]:
dfr['UNDERINFL'].value_counts()

N    82684
0    72240
Y     3804
1     3102
Name: UNDERINFL, dtype: int64

In [92]:
dfr=dfr.loc[dfr['UNDERINFL']!='Y']
dfr=dfr.loc[dfr['UNDERINFL']!='1']
dfr['UNDERINFL'].value_counts()

N    82684
0    72240
Name: UNDERINFL, dtype: int64

Those columns can now be dropped from the dataset.

In [93]:
dfs=dfr[['SEVERITYCODE','INCDTTM','WEATHER','ROADCOND','INATTENTIONIND']].copy()
dfs.head(10)

,SEVERITYCODE,INCDTTM,WEATHER,ROADCOND,INATTENTIONIND
0,2,3/27/2013 2:54:00 PM,Overcast,Wet,NaN
1,1,12/20/2006 6:55:00 PM,Raining,Wet,NaN
2,1,11/18/2004 10:20:00 AM,Overcast,Dry,NaN
3,1,3/29/2013 9:26:00 AM,Clear,Dry,NaN
4,2,1/28/2004 8:04:00 AM,Raining,Wet,NaN
5,1,4/20/2019 5:42:00 PM,Clear,Dry,NaN
6,1,12/9/2008,Raining,Wet,NaN
8,1,6/15/2006 1:00:00 PM,Clear,Dry,NaN
9,2,3/20/2006 3:49:00 PM,Clear,Dry,NaN
10,1,5/25/2006 6:20:00 PM,Overcast,Dry,NaN


In [94]:
dfs.groupby('SEVERITYCODE').count()

,INCDTTM,WEATHER,ROADCOND,INATTENTIONIND
SEVERITYCODE,,,,
1,118143,114463,114497,17109
2,41366,40335,40346,8118


Looking at the data, overcast days in the the weather attribute appear to account for why the road condition attribute contains both more dry and wet days, since there is no value synonymous with overcast. Therefore, to reduce the amount of ambiguity, weather will be used to classify road conditions and further refined to only include clear vs raining days as the other classes do not significantly contribute to the dataset or are unknown.

In [95]:
dfs['ROADCOND'].value_counts()

Dry               102914
Wet                37176
Unknown            13010
Ice                  781
Snow/Slush           693
Other                 91
Standing Water        66
Oil                   60
Sand/Mud/Dirt         52
Name: ROADCOND, dtype: int64

In [96]:
dfs['WEATHER'].value_counts()

Clear                       91367
Raining                     25764
Overcast                    22686
Unknown                     13049
Other                         717
Snowing                       631
Fog/Smog/Smoke                437
Sleet/Hail/Freezing Rain       81
Blowing Sand/Dirt              41
Severe Crosswind               21
Partly Cloudy                   4
Name: WEATHER, dtype: int64

In [97]:
dfs.drop(['ROADCOND'],axis=1,inplace=True)
dfs.head()

,SEVERITYCODE,INCDTTM,WEATHER,INATTENTIONIND
0,2,3/27/2013 2:54:00 PM,Overcast,NaN
1,1,12/20/2006 6:55:00 PM,Raining,NaN
2,1,11/18/2004 10:20:00 AM,Overcast,NaN
3,1,3/29/2013 9:26:00 AM,Clear,NaN
4,2,1/28/2004 8:04:00 AM,Raining,NaN


In [229]:
conditions=['Raining','Clear']
dfw= dfs[dfs.WEATHER.isin(conditions)]
dfw['WEATHER'].value_counts()

Clear      91367
Raining    25764
Name: WEATHER, dtype: int64

In [230]:
dfw.groupby('SEVERITYCODE').count()

,INCDTTM,WEATHER,INATTENTIONIND
SEVERITYCODE,,,
1,83918,83918,13719
2,33213,33213,6777


In [231]:
dfw.head()

,SEVERITYCODE,INCDTTM,WEATHER,INATTENTIONIND
1,1,12/20/2006 6:55:00 PM,Raining,NaN
3,1,3/29/2013 9:26:00 AM,Clear,NaN
4,2,1/28/2004 8:04:00 AM,Raining,NaN
5,1,4/20/2019 5:42:00 PM,Clear,NaN
6,1,12/9/2008,Raining,NaN


In [232]:
dfw['INATTENTIONIND'].value_counts()

Y    20496
Name: INATTENTIONIND, dtype: int64

The NaN values in the INATTENTIONIND need to be converted to a value. 

In [233]:
dfw.INATTENTIONIND = dfw.INATTENTIONIND.fillna('N')
dfw['INATTENTIONIND'].value_counts()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


N    96635
Y    20496
Name: INATTENTIONIND, dtype: int64

The next step is to remove weekend days and split the remaining dataset into specific time intervals available for commuting to and from work based around the current 9am-5pm workday. There will be two potential time intervals: 1) early, for starting and ending the workday two hours earlier, and 2) current, for the current work scedule. Thus, Business X can choose to shift their 8 hour day based on the safest driving times if that proves to be the most significant factor in predicting more severe accidents when compared to weather.

In [234]:
dfw['INCDTTM']=pd.to_datetime(dfw['INCDTTM'])
dfw['DAY']=dfw['INCDTTM'].dt.dayofweek
dfw['HOUR']=dfw['INCDTTM'].dt.hour
dfw.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,SEVERITYCODE,INCDTTM,WEATHER,INATTENTIONIND,DAY,HOUR
1,1,2006-12-20 18:55:00,Raining,N,2,18
3,1,2013-03-29 09:26:00,Clear,N,4,9
4,2,2004-01-28 08:04:00,Raining,N,2,8
5,1,2019-04-20 17:42:00,Clear,N,5,17
6,1,2008-12-09 00:00:00,Raining,N,1,0


In [235]:
dfw.shape

(117131, 6)

In [236]:
dfw['DAY'].value_counts()

4    19571
3    17881
2    17635
1    17258
5    16352
0    15856
6    12578
Name: DAY, dtype: int64

In [237]:
days=[5,6]
dfw= dfw[~dfw.DAY.isin(days)]
dfw['DAY'].value_counts()

4    19571
3    17881
2    17635
1    17258
0    15856
Name: DAY, dtype: int64

In [227]:
dft=dfw[(dfw.HOUR>=5) & (dfw.HOUR<21)]
dft.shape

(67320, 6)

In [238]:
dft.loc[(dft.HOUR>=5)&(dft.HOUR <7),'RUSH']='Early'
dft.loc[(dft.HOUR>=7)&(dft.HOUR <9),'RUSH']='Current'
dft.loc[(dft.HOUR>=9)&(dft.HOUR <11),'RUSH']='Late'
dft.loc[(dft.HOUR >= 15)&(dft.HOUR <17),'RUSH']='Early'
dft.loc[(dft.HOUR >= 17)&(dft.HOUR <19),'RUSH']='Current'
dft.loc[(dft.HOUR >= 19)&(dft.HOUR <21),'RUSH']='Late'
dft.head()

,SEVERITYCODE,INCDTTM,WEATHER,INATTENTIONIND,DAY,HOUR,RUSH
1,1,2006-12-20 18:55:00,Raining,N,2,18,Current
3,1,2013-03-29 09:26:00,Clear,N,4,9,Late
4,2,2004-01-28 08:04:00,Raining,N,2,8,Current
8,1,2006-06-15 13:00:00,Clear,N,3,13,NaN
9,2,2006-03-20 15:49:00,Clear,N,0,15,Early


In [239]:
times=['Early','Current']
dft= dft[dft.RUSH.isin(times)]
dft['RUSH'].value_counts()

Current    19508
Early      14857
Name: RUSH, dtype: int64

In [240]:
dft.groupby(['SEVERITYCODE']).count()

,INCDTTM,WEATHER,INATTENTIONIND,DAY,HOUR,RUSH
SEVERITYCODE,,,,,,
1,23730,23730,23730,23730,23730,23730
2,10635,10635,10635,10635,10635,10635


The INCDTTM, DAY, and HOUR columns can now be dropped from the dataset. 

In [241]:
dft.drop(columns=['INCDTTM','HOUR','DAY'],inplace=True)
dft.head()

,SEVERITYCODE,WEATHER,INATTENTIONIND,RUSH
1,1,Raining,N,Current
4,2,Raining,N,Current
9,2,Clear,N,Early
11,1,Clear,N,Current
13,1,Raining,N,Current


Because the dataset is still unbalanced between the target variable, SEVERITYCOUNT, a random sample of the remaining dataset will be used. 

In [242]:
random1=dft[dft.SEVERITYCODE==1]
random2=dft[dft.SEVERITYCODE==2]
print(random1.shape,"  ",random2.shape)

(23730, 4)    (10635, 4)


In [243]:
random1=random1.sample(n=10000,replace=False,random_state=2,axis=0)
random2=random2.sample(n=10000,replace=False,random_state=2,axis=0)
print(random1.shape,"  ",random2.shape)

(10000, 4)    (10000, 4)


In [244]:
frames = [random1,random2]
dff=pd.concat(frames,ignore_index=True)
print(dff.shape)
dff.groupby(['SEVERITYCODE']).count()

(20000, 4)


,WEATHER,INATTENTIONIND,RUSH
SEVERITYCODE,,,
1,10000,10000,10000
2,10000,10000,10000


In [245]:
dff['SEVERITYCODE'] = dff['SEVERITYCODE'].replace([1],0)
dff['SEVERITYCODE'] = dff['SEVERITYCODE'].replace([2],1)
dff.groupby(['SEVERITYCODE']).count()

,WEATHER,INATTENTIONIND,RUSH
SEVERITYCODE,,,
0,10000,10000,10000
1,10000,10000,10000


The dataset is now ready for modeling.

In [246]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [247]:
X = dff[['WEATHER', 'INATTENTIONIND', 'RUSH']].values
X[0:5]

array([['Clear', 'Y', 'Current'],
       ['Clear', 'N', 'Early'],
       ['Clear', 'N', 'Early'],
       ['Clear', 'Y', 'Current'],
       ['Clear', 'N', 'Current']], dtype=object)

In [248]:
le_w = preprocessing.LabelEncoder()
le_w.fit(['Clear','Raining'])
X[:,0] = le_w.transform(X[:,0]) 

le_in = preprocessing.LabelEncoder()
le_in.fit([ 'N', 'Y'])
X[:,1] = le_in.transform(X[:,1]) 

le_r = preprocessing.LabelEncoder()
le_r.fit([ 'Early', 'Current'])
X[:,2] = le_r.transform(X[:,2])

X[0:5]


array([[0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 0]], dtype=object)

In [249]:
dff['SEVERITYCODE'] = dff['SEVERITYCODE'].astype('int')
y = np.asarray(dff['SEVERITYCODE'])
y [0:5]

array([0, 0, 0, 0, 0])

In [250]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (16000, 3) (16000,)
Test set: (4000, 3) (4000,)


In [251]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=1, solver='liblinear').fit(X_train,y_train)
LR

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [252]:
yhat = LR.predict(X_test)
yhat

array([0, 0, 0, ..., 1, 1, 0])

In [253]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

array([[0.52008459, 0.47991541],
       [0.51501396, 0.48498604],
       [0.51501396, 0.48498604],
       ...,
       [0.45931275, 0.54068725],
       [0.45931275, 0.54068725],
       [0.52008459, 0.47991541]])

In [254]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

0.5105

In [255]:
from sklearn.metrics import classification_report
print (classification_report(y_test, yhat))

              precision    recall  f1-score   support

           0       0.50      0.62      0.55      1980
           1       0.52      0.41      0.46      2020

   micro avg       0.51      0.51      0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000



In [256]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

0.6927201921261861

In [270]:
LRF= LogisticRegression(C=1, solver='liblinear').fit(X,y)

In [271]:
print(LRF.coef_)

[[ 0.06164327  0.21850028 -0.01137967]]


In [272]:
odds=np.exp(LRF.coef_)
odds

array([[1.06358287, 1.24420937, 0.98868483]])

In [273]:
results=pd.DataFrame(odds, columns=['WEATHER', 'INATTENTIONIND', 'RUSH'])
results.rename(index={0: 'Odds'}, inplace=True)
results

,WEATHER,INATTENTIONIND,RUSH
Odds,1.063583,1.244209,0.988685


These results indicate that, although the overall model performs poorly at predicting the severity of an accident, distracted driving is the biggest contributing factor to increasing the odds of a severe accident among the three attributes of weather, distracted driving, and time of day. This is evidenced by the odds ratio that indicates a ~24.4% increased probability of an accident being severe when a driver is distracted compared to an ~6.4% increase when it is raining and, ~1.1% decrease when an accident occurs during an early 2-hour time interval than standard rush hour. Business X will therefore invest in the app that prevents workplace distractions (e.g. email notifications) when employees are detected to be in the car.